In [1]:
import tensorflow as tf
import numpy as np

In [2]:
poem_list = """I'm_ filling
out_ a
filling_ a 
job_ application"""

poem_list = [(poem.split('_ ')[0].strip(), poem.split('_ ')[1].strip()) for poem in poem_list.split('\n')]

poem_list

[("I'm", 'filling'), ('out', 'a'), ('filling', 'a'), ('job', 'application')]

In [3]:
word_list  = [] 
for (title,poet) in poem_list:
    word_list.append(title)
    word_list.append(poet)

word_dic = {value:index for index, value in enumerate(word_list)}

In [4]:
title_dic = {title:word_dic[title] for (title, poet) in poem_list}
title_index = [title_dic[title] for (title, poet) in poem_list]

poet_dic = {poet:word_dic[poet] for (title, poet) in poem_list}
poet_index = [poet_dic[poet] for (title, poet) in poem_list]

In [5]:
poet_name = []

for i in poet_index: 
    for poet, index in poet_dic.items():
        if i == index:
            poet_name.append(poet)

In [6]:
# word = 'hell'
vocab = set(word_list[:])

In [7]:
title_index

[0, 2, 4, 6]

In [8]:
# x_data
x_data = tf.one_hot(title_index, len(vocab), 1.0, 0.0, -1)
y_data = tf.one_hot(poet_index, len(vocab), 1.0, 0.0, -1)
# x_data

In [9]:
title_dic

{"I'm": 0, 'filling': 4, 'job': 6, 'out': 2}

In [10]:
#Configuration

char_vocab_size = len(vocab)
rnn_size = char_vocab_size
time_step_size = len(title_dic)
batch_size = 1

In [11]:
# RNN Model
# from tensorflow.models.rnn import rnn, rnn_cell

one_cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
rnn_cell = tf.nn.rnn_cell.MultiRNNCell([one_cell]*2)
state = tf.zeros([batch_size, rnn_cell.state_size])
X_split = tf.split(0, time_step_size, x_data)
#_split = tf.split(0, time_step_size, X)
outputs, state = tf.nn.rnn(rnn_cell, X_split, state)

In [12]:
print(outputs)
print(state)

[<tf.Tensor 'RNN/MultiRNNCell/Cell1/BasicRNNCell/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_1/Cell1/BasicRNNCell/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_2/Cell1/BasicRNNCell/Tanh:0' shape=(1, 6) dtype=float32>, <tf.Tensor 'RNN/MultiRNNCell_3/Cell1/BasicRNNCell/Tanh:0' shape=(1, 6) dtype=float32>]
Tensor("RNN/concat_3:0", shape=(1, 12), dtype=float32)


In [13]:
rnn_size, time_step_size

(6, 4)

In [14]:
logits = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
targets = tf.reshape(poet_index, [-1])
weights = tf.ones(time_step_size*batch_size)

In [15]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])


In [16]:
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

In [17]:
#Launch the graph in session

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(1000):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        if i%40 == 0:
            print(result, [word_list[t] for t in result])

        W_val = sess.run(train_op)

[5 5 1 5] ['a', 'a', 'filling', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 4] ['filling', 'a', 'a', 'filling']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 3] ['filling', 'a', 'a', 'a']
[4 5 5 5] ['filling', 'a', 'a', 'a']
[4 5 5 4] ['filling', 'a', 'a', 'filling']
[4 5 5 4] ['filling', 'a', 'a', 'filling']
[4 5 5 3] ['filling', 'a', 'a', 'a']
[4 5 5 4] ['filling', 'a', 'a', 'filling']
[4 5 5 5] ['filling', 'a', 'a', 'a']


In [18]:
[result[i] == poet_index[i] for i in range(len(poet_index))]

[True, True, True, False]